# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

![diagnoses](intro.png "Diagnoses")

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [1]:
from scipy import stats
import pandas as pd
import numpy as np

import statsmodels.stats.multitest as smm

In [27]:
def fc(c, t):
    return t/c if t>=c else -c/t

In [2]:
df = pd.read_csv("./gene_high_throughput_sequencing.csv")

In [3]:
genes = list(filter(lambda x: x not in {u'Patient_id', u'Diagnosis'}, df.columns))
print len(genes)

15748


## Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)
2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

### 1.1 Проверка для групп normal (control) и early neoplasia (treatment)

In [ ]:
d_control = df.loc[df.Diagnosis == "normal", :]
d_treatment = df.loc[df.Diagnosis == "early neoplasia", :]

result = []
for gene in genes:
    res = stats.ttest_ind(d_control[gene], d_treatment[gene], equal_var=False)
    if res.pvalue<0.05:
        result.append((gene, res.pvalue))
    elif res.pvalue==0.05:
        print gene

print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05"%len(result)

with open("work_1_1.txt",'w') as f:
    f.write(str(len(result)))

### 1.2 Проверка для групп early neoplasia (control) и cancer (treatment)

In [ ]:
d_control = df.loc[df.Diagnosis == "early neoplasia", :]
d_treatment = df.loc[df.Diagnosis == "cancer", :]

result = []
for gene in genes:
    res = stats.ttest_ind(d_control[gene], d_treatment[gene], equal_var=False)
    if res.pvalue<0.05:
        result.append((gene, res.pvalue))
    elif res.pvalue==0.05:
        print gene

print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05"%len(result)

with open("work_1_2.txt",'w') as f:
    f.write(str(len(result)))

## Часть 2: поправка методом Холма



### 2.1 Проверка для групп normal (control) и early neoplasia (treatment)

In [ ]:
d_control = df.loc[df.Diagnosis == "normal", :]
d_treatment = df.loc[df.Diagnosis == "early neoplasia", :]

obs = []
for gene in genes:
    res = stats.ttest_ind(d_control[gene], d_treatment[gene], equal_var=False)
    obs.append(
        (
            gene, 
            res.pvalue,
            fc(d_control[gene].mean(), d_treatment[gene].mean())
        )
    )

pv = map( lambda x: x[1], obs)
rejected, _, _, _ = smm.multipletests(pv, alpha=.025, method='holm')

result = 0
for i in range(len(obs)):
    if abs(obs[i][2]) > 1.5 and rejected[i]:
        result += 1
        
print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05"%result

with open("work_2_1.txt",'w') as f:
    f.write(str(result))

### 2.2 Проверка для групп early neoplasia (control) и cancer (treatment)

In [28]:
d_control = df.loc[df.Diagnosis == "early neoplasia", :]
d_treatment = df.loc[df.Diagnosis == "cancer", :]

obs = []
for gene in genes:
    res = stats.ttest_ind(
        d_control[gene], 
        d_treatment[gene], 
        equal_var=False
    )
    obs.append(
        (
            gene, 
            res.pvalue,
            fc(d_control[gene].mean(), d_treatment[gene].mean())
        )
    )

pv = map(lambda x: x[1], obs)
rejected, _, _, _ = smm.multipletests(pv, alpha=.025, method='holm')

result = 0
for i in range(len(obs)):
    if abs(obs[i][2]) >= 1.5 and rejected[i]:
        result += 1

print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05" % result

with open("work_2_2.txt",'w') as f:
    f.write(str(result))


У 77 генов отвергается нулевая гипотеза с уровнем значимости 0.05


## Часть 3: поправка методом Бенджамини-Хохберга

### 3.1 Проверка для групп normal (control) и early neoplasia (treatment)

In [33]:
d_control = df.loc[df.Diagnosis == "normal", :]
d_treatment = df.loc[df.Diagnosis == "early neoplasia", :]

obs = []
for gene in genes:
    res = stats.ttest_ind(d_control[gene], d_treatment[gene], equal_var=False)
    obs.append(
        (
            gene, 
            res.pvalue,
            fc(d_control[gene].mean(), d_treatment[gene].mean())
        )
    )

pv = map( lambda x: x[1], obs)
rejected, _, _, _ = smm.multipletests(pv, alpha=.025, method='fdr_bh')

result = 0
for i in range(len(obs)):
    if abs(obs[i][2]) > 1.5 and rejected[i]:
        result += 1
        
print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05"%result

with open("work_3_1.txt",'w') as f:
    f.write(str(result))

У 4 генов отвергается нулевая гипотеза с уровнем значимости 0.05


### 3.2 Проверка для групп early neoplasia (control) и cancer (treatment)

In [35]:
d_control = df.loc[df.Diagnosis == "early neoplasia", :]
d_treatment = df.loc[df.Diagnosis == "cancer", :]

obs = []
for gene in genes:
    res = stats.ttest_ind(d_control[gene], d_treatment[gene], equal_var=False)
    obs.append(
        (
            gene, 
            res.pvalue,
            fc(d_control[gene].mean(), d_treatment[gene].mean())
        )
    )

pv = map( lambda x: x[1], obs)
rejected, _, _, _ = smm.multipletests(pv, alpha=.025, method='fdr_bh')

result = 0
for i in range(len(obs)):
    if abs(obs[i][2]) > 1.5 and rejected[i]:
        result += 1
        
print "У %s генов отвергается нулевая гипотеза с уровнем значимости 0.05"%result

with open("work_3_2.txt",'w') as f:
    f.write(str(result))

У 524 генов отвергается нулевая гипотеза с уровнем значимости 0.05
